In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

# Replace '?' with NaN
merged_dataset.replace('?', np.nan, inplace=True)

# Convert all columns to numeric
merged_dataset = merged_dataset.apply(pd.to_numeric, errors='coerce')

# Drop rows where the target variable is NaN
merged_dataset.dropna(subset=['target'], inplace=True)  # Replace 'target' with your actual target column name

# Define features and target variable
X = merged_dataset.drop('target', axis=1)  # Replace 'target' with your actual target column name
y = merged_dataset['target']  # Replace 'target' with your actual target column name

# Impute missing values in features using the mean of each column
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# Initialize the Neural Network classifier
model2 = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)

# Train the model
model2.fit(X_train, y_train)

# Make predictions
y_pred = model2.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Neural Network Model Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# prompt: predict using a record

import numpy as np
# Example new record (replace with your actual values)
new_record = np.array([[67, 0, 3, 60, 200, 0, 0, 100, 0, 1.6, 0, 0, 7]])

# Impute missing values if any (using the same imputer fitted earlier)
new_record_imputed = imputer.transform(new_record)

# Make prediction
prediction = model2.predict(new_record_imputed)

# Print prediction
if prediction[0] == 0:
    print("Prediction: No Heart Attack")
else:
    print("Prediction: Heart Attack")
